# Porting genome scale metabolic models for metabolomics
- from ModelSeed

**source**
- https://modelseed.org/

**Use cobra to parse SBML models whereas applicable**

Not all models comply with the formats in cobra. Models from USCD and Thiele labs should comply.

**Base our code on metDataModel**

Each model needs a list of Reactions, list of Pathways, and a list of Compounds.
It's important to include with Compounds with all linked identifiers to other DBs (HMDB, PubChem, etc), and with formulae (usually charged form in these models) when available.
We can alwasy update the data later. E.g. the neural formulae can be inferred from charged formula or retrieved from public metabolite database (e.g., HMDB) if linked.
Save in Python pickle and in JSON.

**No compartmentalization**
- After decompartmentalization,
  - transport reactions can be removed - they are identified by reactants and products being the same.
  - redundant reactions can be merge - same reactions in diff compartments become one.

Minghao Gong, 2022-04-26

In [205]:
# !pip install cobra --user --ignore-installed ruamel.yaml
# !pip install --upgrade metDataModel # https://github.com/shuzhao-li/metDataModel/ 
# !pip install --upgrade numpy pandas

In [206]:
import cobra # https://cobrapy.readthedocs.io/en/latest/io.html#SBML
from metDataModel.core import Compound, Reaction, Pathway, MetabolicModel
import requests
import sys
import re

sys.path.append("/Users/gongm/Documents/projects/mass2chem/")
sys.path.append("/Users/gongm/Documents/projects/JMS/JMS/JMS")
from mass2chem.formula import *
from jms.formula import *
from jms.utils.gems import *
from jms.utils.git_download import *

In [207]:
# download the most updated Rat-GEM.xml
model_name = 'Clostridium_sporogenes-modelseed'
file_name = 'Clostridium_sporogenes_modelseed.sbml'
local_path = output_fdr = '../testdata/draftGenome-GEM-JMS/ModelSeed/'

In [208]:
# Read the model via cobra
model = cobra.io.read_sbml_model(os.path.join(local_path,file_name))

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00443_c0 "ABEE_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd02920_c0 "2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydropteridinediphosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00012_c0 "PPi_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00067_c0 "H_plus__c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00683_c0 "Dihydropteroate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00114_c0 "IMP_c0">
Use of the species charge attribute is discouraged, use fbc:charge

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd15735_c0 "Diglucosyl_1_2_dianteisopentadecanoylglycerol_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd08739_c0 "trans_2_Methyl_5_isopropylhexa_2_5_dienal_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd08740_c0 "trans_2_Methyl_5_isopropylhexa_2_5_dienoic_acid_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd17042_c0 "DNA_replication_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd11492_c0 "Malonyl_acyl_carrierprotein__c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd11528_c0 "7_methyl_octanoyl_ACP_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd11529_c0 "9_methyl_3_oxo_decanoyl_ACP_c0">
Use of the species charge attribut

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd03246_c0 "Taurochenodeoxycholate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00210_c0 "Taurine_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd01663_c0 "Chenodiol_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00080_c0 "Glycerol_3_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd15686_c0 "CDP_1_2_diisopentadecanoylglycerol_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd15719_c0 "Diisopentadecanoylphosphatidylglycerophosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd02005_c0 "DMF_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00047_c0 "Formate_c0">
Use 

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00424_c0 "Cystathionine_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd15308_c0 "1_2_Diacyl_sn_glycerol_ditetradec_7_enoyl_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd15523_c0 "1_2_ditetradec_7_enoyl_sn_glycerol_3_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd03396_c0 "Selenate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd03388_c0 "Adenylylselenate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd01030_e0 "Salicin_e0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd03698_c0 "Salicin_6P_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00309_c0 "XAN_c0">
Use of t

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd03917_c0 "Adenosylcobyric_acid_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd03919_c0 "Adenosyl_cobinamide_phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00652_c0 "D_Mucic_acid_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00515_c0 "5_Dehydro_4_deoxy_D_glucarate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00689_c0 "Porphobilinogen_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00755_c0 "Hydroxymethylbilane_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00196_c0 "Acetylphosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd11472_c0 "Hexanoyl_ACP_c0">
U

...

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn05425_c0 "10_methyl_undecanoyl_ACP_malonyl_acyl_carrier_protein_C_acyltransferase_decarboxylating_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn05736_c0 "acyl_coenzyme_A_ligase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn10318_c0 "stearoyl_lipoteichoic_acid_synthesis_n24__unlinked__D_alanine_substituted_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn01641_c0 "5_Formiminotetrahydrofolate_L_glutamate_N_formiminotransferase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn01226_c0 "Guanosine_ribohydrolase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn00493_c0 "L

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn12851_c0 "Gly_Try_ABC_transporters_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn05437_c0 "12_methyl_trans_tetra_dec_2_enoyl_ACP_NAD_plus__oxidoreductase_A_specific_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn04139_c0 "2_Octaprenyl_3_methyl_6_methoxy_1_4_benzoquinone__NADPH2_oxygen_oxidoreductase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn03393_c0 "3_octaprenyl_4_hydroxybenzoate_carboxy_lyase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn10094_c0 "S_adenosylmethione_2_demethylmenaquinone_methyltransferase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn0

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn08040_c0 "UDP_N_acetyl_D_glucosamine_undecaprenyl_phosphate_N_acetyl_D_glucosamine_phosphotransferase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn10215_c0 "anteisoheptadecanoyl_1_acylglycerol_3_phosphate_O_acyltransferase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn05148_c0 "Heme_transport_via_ABC_system_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn09128_c0 "Phospholipase_A1_phosphatidate__n_C18_0_periplasm_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_rxn03893_c0 "all_trans_octaprenyl_diphosphate_4_hydroxybenzoate_3_octaprenyltransferase_c0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObje

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd00438_e0 "EX_Deoxyadenosine_e0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd00034_e0 "EX_Zn2_plus__e0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd00249_e0 "EX_Uridine_e0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd00654_e0 "EX_Deoxycytidine_e0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd00076_e0 "EX_Sucrose_e0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd11579_e0 "EX_ethanesulfonate_e0">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd01171_e0 "EX_Dulcose_e0">
Encoding OBJECTIVE_COEFFICIENT in Kinet

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction EX_cpd00111_e0 "EX_Oxidized_glutathione_e0">


In [209]:
model

Name,Clostridium_sporogenes_modelseed
Memory address,0x07fef21ef8e50
Number of metabolites,1335
Number of reactions,1246
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol_0, Extracellular_0"


In [210]:
# metabolite entries, readily convert to list of metabolites
model.metabolites[33]

Metabolite identifier,cpd00096_c0
Name,CDP_c0
Memory address,0x07fef0777c4c0
Formula,None
Compartment,c0
In 2 reaction(s),"rxn06076_c0, rxn00364_c0"


In [211]:
# reaction entries, Readily convert to list of reactions
model.reactions[500]

Reaction identifier,rxn01509_c0
Name,ATP_dGMP_phosphotransferase_c0
Memory address,0x07fef1caa86d0
Stoichiometry,cpd00002_c0 + cpd00067_c0 + cpd00296_c0 <=> cpd00008_c0 + cpd00295_c0 ATP_c0 + H_plus__c0 + dGMP_c0 <=> ADP_c0 + dGDP_c0
GPR,(fig or 6666666.917722.peg.2435) or (fig or 6666666.917722.peg.2435)
Lower bound,-1000.0
Upper bound,1000.0


In [212]:
# NO group or pathway information
# model.groups[33]

In [213]:
model.metabolites[100]

Metabolite identifier,cpd02666_c0
Name,R_4_Phosphopantothenoyl_L_cysteine_c0
Memory address,0x07fef01a11a00
Formula,None
Compartment,c0
In 3 reaction(s),"rxn02341_c0, rxn09177_c0, rxn12512_c0"


## Port metabolite

In [214]:
def port_metabolite(M):
    # convert cobra Metabolite to metDataModel Compound
    Cpd = Compound()
    Cpd.src_id = remove_compartment_by_split(M.id,'_')
    Cpd.id = remove_compartment_by_split(M.id,'_')              # temporarily the same with the source id
    Cpd.name = M.name
    Cpd.charge = M.charge
#     Cpd.neutral_formula = adjust_charge_in_formula(M.formula,M.charge) There are no formula of charge formula
#     Cpd.neutral_mono_mass = neutral_formula2mass(Cpd.neutral_formula)
    Cpd.charged_formula = M.formula
    Cpd.db_ids = [[model_name,Cpd.src_id]] # using src_id to also reference Rat-GEM ID in db_ids field
    for k,v in M.annotation.items():
        if k != 'sbo':
            if isinstance(v,list):
                Cpd.db_ids.append([[k,x] for x in v])
            else: 
                if ":" in v:
                    Cpd.db_ids.append([k,v.split(":")[1]])
                else:
                    Cpd.db_ids.append([k,v])
    
    inchi_list = [x[1].split('=')[1] for x in Cpd.db_ids if x[0] == 'inchi']
    if len(inchi_list) ==1:
        Cpd.inchi = inchi_list[0]
    elif len(inchi_list) >1:
        Cpd.inchi = inchi_list
        
    return Cpd

In [215]:
myCpds = []
for i in range(len(model.metabolites)):
    myCpds.append(port_metabolite(model.metabolites[i]))

In [216]:
len(myCpds)

1335

In [217]:
# remove duplicated compounds
myCpds = remove_duplicate_cpd(myCpds)

In [218]:
len(myCpds)

1164

In [219]:
model.metabolites[100].__dict__

{'_id': 'cpd02666_c0',
 'name': 'R_4_Phosphopantothenoyl_L_cysteine_c0',
 'notes': {},
 '_annotation': {},
 '_model': <Model Clostridium_sporogenes_modelseed at 0x7fef21ef8e50>,
 '_reaction': {<Reaction rxn02341_c0 at 0x7fef0e929d00>,
  <Reaction rxn09177_c0 at 0x7fef01595eb0>,
  <Reaction rxn12512_c0 at 0x7fef0e97eaf0>},
 'formula': None,
 'compartment': 'c0',
 'charge': -3,
 '_bound': 0.0}

In [220]:
myCpds[100].__dict__['id']

'cpd03524'

In [221]:
myCpds[100].__dict__

{'internal_id': '',
 'id': 'cpd03524',
 'name': '10_Formyl_THF_L_glutamate_c0',
 'db_ids': [['Clostridium_sporogenes-modelseed', 'cpd03524']],
 'neutral_formula': '',
 'neutral_mono_mass': 0.0,
 'charge': -3,
 'charged_formula': None,
 'SMILES': '',
 'inchi': '',
 'src_id': 'cpd03524'}

## Port reactions

In [222]:
[x.id for x in model.reactions[190].genes]

['6666666.917722.peg.451',
 '6666666.917722.peg.456',
 '6666666.917722.peg.2371',
 'fig']

In [223]:
# port reactions, to include genes and enzymes
def port_reaction(R):
    new = Reaction()
    new.id = remove_compartment_by_split(R.id,'_')
    new.genes = [g.id for g in R.genes]
    # ecs = R.annotation.get('ec-code', [])
    if 'EX_' not in new.id:
        return new
test99 = port_reaction(model.reactions[250])

In [224]:
model.reactions[250].__dict__

{'_id': 'rxn08802_c0',
 'name': 'Lysophospholipase_L1_2_acylglycerophosphotidate__n_C18_1_periplasm_c0',
 'notes': {'GENE_ASSOCIATION': '(fig|6666666.917722.peg.657 or fig|6666666.917722.peg.788 or fig|6666666.917722.peg.789)',
  'PROTEIN_ASSOCIATION': '(fig|6666666.917722.peg.657 or fig|6666666.917722.peg.788 or fig|6666666.917722.peg.789)'},
 '_annotation': {},
 '_gene_reaction_rule': '(fig or 6666666.917722.peg.657) or (fig or 6666666.917722.peg.788) or (fig or 6666666.917722.peg.789)',
 'subsystem': '',
 '_genes': {<Gene 6666666.917722.peg.657 at 0x7fef138d82e0>,
  <Gene 6666666.917722.peg.788 at 0x7fef138d8220>,
  <Gene 6666666.917722.peg.789 at 0x7fef138d8280>,
  <Gene fig at 0x7fef0e48baf0>},
 '_metabolites': {<Metabolite cpd00001_c0 at 0x7fef0777a3a0>: -1.0,
  <Metabolite cpd15328_c0 at 0x7fef13979940>: -1.0,
  <Metabolite cpd00067_c0 at 0x7fef0777a790>: 2.0,
  <Metabolite cpd00080_c0 at 0x7fef01a17fa0>: 1.0,
  <Metabolite cpd15269_c0 at 0x7fef1c754df0>: 1.0},
 '_model': <Model

In [225]:
model.reactions[99].id

'rxn00364_c0'

In [226]:
port_reaction(model.reactions[99])

In [227]:
## Reactions to port
myRxns = []
for R in model.reactions:
    temp_rxn = port_reaction(R) 
    if temp_rxn:
        myRxns.append(temp_rxn)
    
print(len(myRxns))

1064


In [228]:
len(myRxns)

1064

In [229]:
myRxns[0].__dict__

{'azimuth_id': '',
 'id': 'rxn02201',
 'source': [],
 'version': '',
 'status': '',
 'reactants': [],
 'products': [],
 'enzymes': [],
 'genes': ['6666666.917722.peg.3429', '6666666.917722.peg.1003', 'fig'],
 'pathways': [],
 'ontologies': [],
 'species': '',
 'compartments': [],
 'cell_types': [],
 'tissues': []}

## Pathway information will be parsed later when looking back to the original data

## Collected data; now output

In [230]:
from datetime import datetime
today =  str(datetime.today()).split(" ")[0]

In [231]:
today

'2022-04-26'

In [232]:
note = """modelSeed decompartmentalized, with genes and ECs."""

## metabolicModel to export
MM = MetabolicModel()
MM.id = f'az_{model_name}_{today}' #
MM.meta_data = {
            'species': model_name.split('-')[0],
            'version': '',
            'sources': [f'modelSeed, retrieved {today}'], #
            'status': '',
            'last_update': today,  #
            'note': note,
        }

MM.list_of_reactions = [R.serialize() for R in  myRxns]
MM.list_of_compounds = [C.serialize() for C in myCpds]

In [233]:
import json
with open('../test/output/ModelSeed/Universal_ModelSeed.json', 'r') as f:
    modelSeed_uni_model = json.load(f)

In [242]:
import warnings
def fetch_ModelSeed_GEM_info(ModelSeed_model, # serialized
                             local_path,
                             universal_json_name = 'Universal_ModelSeed.json',
                             overwrite = True,
                             download = False):
    if download:
        url_met = f'https://github.com/shuzhao-li/JMS/blob/main/jms/data/staged/Universal_ModelSeed.json' # may change
        git_download_from_file(url_met,local_path,metab_file_name)
    
    with open(os.path.join(local_path,universal_json_name), 'r') as f:
        uni_model = json.load(f)
    
    # Replacing the list of compound information
    uniCpd_dict = {}
    for uniCpd in uni_model['list_of_compounds']:
        uniCpd_dict.update({uniCpd['id']:uniCpd})
    new_list_of_compounds = []
    for myCpd in ModelSeed_model.list_of_compounds:
        try:
            new_list_of_compounds.append(uniCpd_dict[myCpd['id']])
        except:
            warnings.warn(f"not mapping the {myCpd['id']}")
    ModelSeed_model.list_of_compounds = new_list_of_compounds
            

    # Replace the list of reaction information
    uniRxn_dict = {}
    for uniRxn in uni_model['list_of_reactions']:
        uniRxn_dict.update({uniRxn['id']:uniRxn})
    new_list_of_reactions = []
    for myRxn in ModelSeed_model.list_of_reactions:
        try:
            genes = myRxn['genes']
            myRxn = uniRxn_dict[myRxn['id']]
            myRxn['genes'] = genes # keep genes 
            new_list_of_reactions.append(myRxn)
        except:
            warnings.warn(f"not mapping the {myRxn['id']}")
    ModelSeed_model.list_of_reactions = new_list_of_reactions
    
    # gathering subsystem information
    uniPty_dict = {} 
    ModelSeed_model.list_of_pathways = []
    for uniPty in uni_model['list_of_pathways']:
        temp = set(uniPty['list_of_reactions']).intersection(set([x['id'] for x in ModelSeed_model.list_of_reactions]))
        if len(temp)>0:
            print("yes")
            uniPty['list_of_reactions'] = list(temp)
            ModelSeed_model.list_of_pathways.append(uniPty)
    return ModelSeed_model

In [243]:
test = fetch_ModelSeed_GEM_info(MM, # serialized
                             local_path = '../test/output/ModelSeed/',
                             universal_json_name = 'Universal_ModelSeed.json',
                             overwrite = True,
                             download = False)

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [247]:
# check output
[
MM.list_of_pathways[2],
MM.list_of_reactions[:2],
MM.list_of_compounds[100:102],
]

[{'id': 'MSPwy0003',
  'name': 'A cluster perhaps related to purine metabolism',
  'list_of_reactions': ['rxn00800', 'rxn03136', 'rxn13784']},
 [{'id': 'rxn02201',
   'reactants': ['cpd00443', 'cpd02920'],
   'products': ['cpd00012', 'cpd00683'],
   'genes': ['6666666.917722.peg.3429', '6666666.917722.peg.1003', 'fig'],
   'enzymes': ['2.5.1.15']},
  {'id': 'rxn00836',
   'reactants': ['cpd00012', 'cpd00114'],
   'products': ['cpd00103', 'cpd00226'],
   'genes': ['6666666.917722.peg.533', '6666666.917722.peg.1813', 'fig'],
   'enzymes': ['2.4.2.8']}],
 [{'id': 'cpd03524',
   'name': '(6R)-10-Formyltetrahydropteroyldiglutamate; (6R)-10-HCO-H4PteGlu2; 10-Formyl-THF-L-glutamate; 10-Formyltetrahydrofolyl L-glutamate; 10-formyl-H4PteGlu2; 10-formyl-THF di-L-glutamate; 10-formyl-tetrahydrofolate di-L-glutamate; N10-formyl-H4F di-L-glutamate; N10-formyl-H4PteGlu2; N10-formyl-THF di-L-glutamate; N10-formyl-tetrahydrofolate di-L-glutamate',
   'identifiers': [['BiGG', '10fthfglu__L'],
    ['JP_

In [248]:
import pickle
import os

# Write pickle file
export_pickle(os.path.join(output_fdr,f'{MM.id}.pickle'), MM)

In [249]:
# Write json file
export_json(os.path.join(output_fdr,f'{MM.id}.json'), MM)

In [250]:
# Write dataframe 
import pandas as pd
export_table(os.path.join(output_fdr,f'{MM.id}_list_of_compounds.csv'),MM, 'list_of_compounds')
export_table(os.path.join(output_fdr,f'{MM.id}_list_of_reactions.csv'),MM, 'list_of_reactions')
export_table(os.path.join(output_fdr,f'{MM.id}_list_of_pathways.csv'),MM, 'list_of_pathways')

## Summary

This ports reactions, pathways and compounds. Gene and enzyme information is now included. 

The exported pickle can be re-imported and uploaded to Database easily.

This notebook, the pickle file and the JSON file go to GitHub repo (https://github.com/shuzhao-li/Azimuth).